In [361]:
import pandas as pd
import string
from string import punctuation
# import spacy
# Spacy_Eng = spacy.load('en_core_web_sm')
# Spacy_Eng_STOP_WORDS = Spacy_Eng.Defaults.stop_words
import re
import nltk
from nltk.stem.porter import PorterStemmer
p_stemmer = PorterStemmer()
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
import itertools
from collections import defaultdict

In [417]:
df = pd.read_csv("walmart-products-modified.csv", encoding='latin1') # df

In [395]:
df.head(2) # df

,name,rating,type_of,price,comments
0,"Eliminator Ant, Roach & Spider Killer4, 20 oz,...",4.084,"Pest Bait, Traps & Killers",3.48,Covered one up in phone. The roach crawled out...
1,"Spiderman Party Blowers, 8ct",NaN,Novelty & Gag Toys,3.74,NaN


In [499]:
terms_dictionary_for_zone = {
        'name': { },
         'type_of': { },
         'comments': { }
         }

In [501]:
def stemmed_inverted_index_from_data(
    doc_database # list[str,]
):
    for zone in doc_database.columns:
        if zone in terms_dictionary_for_zone.keys():
            # terms_dictionary_for_zone = {{}}
            for docID, doc in enumerate(df[zone]): # int, str; list[(int, str), ]

                sentence_doc = word_tokenize(str(doc)) # list[str,];
                # stems all words in doc
                sentence_str_1 = [p_stemmer.stem(word_tk) for word_tk in sentence_doc]  # list[str,]
                sentence_str_1 = " ".join(sentence_str_1) # str
                # subs all symbols/punctuations (including space) with a space (' '); excludes digits
                sentence_str_1 = re.sub(r'\W+', ' ',sentence_str_1)
                # subs all digits with a space (' '); a sequence of digits (without space) is given one (' ')
                sentence_str_1 = re.sub(r'\d+', ' ',sentence_str_1)
                # subs all symbols AND digits (including spaces) with a space (' '); includes digits
                sentence_str_1 = re.sub(r'[^a-zA-Z]', ' ',sentence_str_1)
                # compiles a pattern and subs out the pattern in sentence_str_1 with ' '
                sentence_str_1 = re.compile('(\s*)pron(\s*)').sub(' ', sentence_str_1)

                sentence_doc_1 = word_tokenize(sentence_str_1) # list[str,]
                # strip removes whitespace at beginning and end of string (not in between)
                sentence_str_2 = [str(word_tk_1).strip() for word_tk_1 in sentence_doc_1] # list[str,]
                # strings with >=2 chars only
                sentence_str_2  = [word_str for word_str in sentence_str_2 if len(word_str) >= 2] # list[str,]
                # strings which are not symbols/punctuations
                sentence_str_2 = [word_str for word_str in sentence_str_2  if word_str not in punctuation]
                # strings which are not stop-words; check with: Spacy_Eng.vocab['word'].is_stop
                sentence_str_2 = [word_str for word_str in sentence_str_2 if word_str not in stop_words]

                for term in sentence_str_2: # 'str'
                    if term in terms_dictionary_for_zone[zone]:
                        terms_dictionary_for_zone[zone][term][0] += 1 # int; doc freq
                        terms_dictionary_for_zone[zone][term][1].append(docID) # list[int,]
                    else:
                        terms_dictionary_for_zone[zone][term] =[[],[]] # list[[int], [int, ]]; list[[doc freq], [docIds,]]
                        terms_dictionary_for_zone[zone][term][0] = 1 # int; doc freq
                        terms_dictionary_for_zone[zone][term][1].append(docID)
    return terms_dictionary_for_zone

In [346]:
# Test case
# doc1 = {
#     'name': 'Fuji',
#     'rating': '4.5',
#     'type_of': 'apple',
#     'category': 'fruit',
#     'price': '2.50',
#     'comments': 'this is an apple, it is kind of fresh but also small, smaller than a grape'
# }
# doc2 = {
#     'name': 'mcIntosh',
#     'rating': '3.6',
#     'type_of': 'apple',
#     'category': 'fruit',
#     'price': '3.50',
#     'comments': 'it was not as sweet as i hoped'
# }
# doc3 = {
#     'name': 'concord',
#     'rating': '4.8',
#     'type_of': 'grape',
#     'category': 'fruit',
#     'price': '4.50',
#     'comments': 'trying grapes for the first time, not too bad'
# }
# doc4 = {
#     'name': 'Red anjou',
#     'rating': '4.5',
#     'type_of': 'pear',
#     'category': 'fruit',
#     'price': '1.50',
#     'comments': 'the shape of the peach is appealing, haha'
# }


# doc_database_list = [doc1, doc2, doc3, doc4] # list[dict, ]
# doc_database = defaultdict(list) # defaultdict({key: list[str, ]})
# for doc in doc_database_list:
#     for key, val in doc.items():
#         doc_database[key].append(val)
# doc_database

In [502]:
inverted_index_zonal_dictionary = stemmed_inverted_index_from_data(df)

In [503]:
def query_processing(
    query # str
):
    posting_list = [] # list[]
    # posting_list_combined_for_and = [] #list[]
    # posting_list_combined_for_or = [] # list[]
    query_terms = query.split() #list[str, ]
    each_term = 0 # int

    for term in query_terms[:]: # str       

        edit_dist_list = [[key, nltk.edit_distance(term, key)]  for key in inverted_index_zonal_dictionary['name']] # list[[str, double], ]
        edit_dist_list_1 = list(zip(*edit_dist_list)) # list[(str,)(double,)]
        internal_query = edit_dist_list_1[0][edit_dist_list_1[1].index(min(edit_dist_list_1[1]))] # str

        posting_list.append(list(inverted_index_zonal_dictionary['name'][internal_query][1])) # list[[int, ], ]
        each_term +=1
          
    posting_list_combined_for_and_high_priority = set.intersection(*map(set, posting_list)) # set[int, ]
    # for docID_of_cur_term in posting_list_combined_for_and_high_priority:
    #     print(doc_database[docID_of_cur_term])
    # return posting_list_combined_for_and_high_priority
    posting_list_combined_for_and_high_priority = list(posting_list_combined_for_and_high_priority) # list[int, ]

    
    posting_list_combined = list(itertools.chain(*posting_list)) # list[int, ]
    posting_list_combined_for_or_low_priority = set(posting_list_combined) # set[int, ]
    posting_list_combined_for_or_low_priority = list(posting_list_combined_for_or_low_priority)

    for docID_low_priority in posting_list_combined_for_or_low_priority: # int
        if docID_low_priority not in posting_list_combined_for_and_high_priority:
            posting_list_combined_for_and_high_priority.append(docID_low_priority) # list[int,]

    for docID_of_cur_term in posting_list_combined_for_and_high_priority:
        print(pd.DataFrame(df.loc[docID_of_cur_term]))
    return posting_list_combined_for_and_high_priority

In [504]:
query = 'h0t sht' # 'str' typo; on both terms
all_postings = query_processing(query)


                                                        163
name      Hot Shot 17.5 Oz. Lemon Fresh Scent Aerosol Sp...
rating                                                    0
type_of                          Pest Bait, Traps & Killers
price                                                  8.79
comments                                                NaN
                                                          3
name      Hot Shot Fresh Floral Scent Ant, Roach & Spide...
rating                                               4.1032
type_of                          Pest Bait, Traps & Killers
price                                                  6.36
comments       Sprayed them directly 10 times. did nothing!
                                                        131
name      Hot Shot Spider and Scorpion Killer Aerosol, 1...
rating                                                  NaN
type_of                          Pest Bait, Traps & Killers
price                                   

In [492]:
all_postings

[163, 3, 131, 101, 139, 14, 245, 603, 413, 515, 619, 143, 366, 154, 158, 351]